In [2]:
import os

import pymongo
from bson.json_util import dumps, loads
from bson.objectid import ObjectId

from splinter import Browser
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait # available since 2.4.0
from selenium.webdriver.support import expected_conditions as EC # available since 2.26.0
from selenium.common.exceptions import TimeoutException

import pandas as pd

import re
from pprint import pprint
from datetime import datetime as dt
import time
import isodate

import sys
# sys.path.append('C:\\Users\\soria\\Documents\\WashUDataDocuments\\HwActivites\\YouTube_Project\\')

import pandas as pd
import json
import os
from pprint import pprint

In [3]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
mongoDb = client.good_eats
collection = mongoDb.ge_scrape

In [38]:
executable_path = {'executable_path': '/Users/soria/Anaconda3/Scripts/chromedriver'}
# browser = Browser('chrome', **executable_path)
driver = webdriver.Chrome()

In [5]:
url = 'http://www.goodeatsfanpage.com/gefp/episodebyorder.htm'
driver.get(url)

In [6]:

html = driver.page_source
ge_soup = bs(html, 'html.parser')


In [13]:
ge_soup

<html xmlns="http://www.w3.org/1999/xhtml"><head>
<meta content="en-us" http-equiv="Content-Language"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="Microsoft FrontPage 6.0" name="GENERATOR"/>
<meta content="FrontPage.Editor.Document" name="ProgId"/>
<title>Good Eats Show Index</title><meta content="postmodern2 0111" name="Microsoft Theme"/>
<link as="script" href="https://adservice.google.com/adsid/integrator.js?domain=www.goodeatsfanpage.com" rel="preload"/><script async="" src="//www.google-analytics.com/analytics.js"></script><script src="https://www.googletagservices.com/activeview/js/current/osd.js?cb=%2Fr20100101"></script><script src="https://pagead2.googlesyndication.com/pub-config/r20160913/ca-pub-.js"></script><script src="https://adservice.google.com/adsid/integrator.js?domain=www.goodeatsfanpage.com" type="text/javascript"></script><link as="script" href="http://pagead2.googlesyndication.com/pagead/js/r20190214/r20190131/show_ads_impl

In [16]:
trans = ge_soup.findAll('td', {'width': '59'})
len(trans)

267

In [36]:
tran_links = []
for tran in trans[1:267]:
    try: 
        raw = tran.find('a')['href'][2:]
#         print(raw)

        if raw[0:2] == '..':
            raw.replace('..', '')
            url = 'http://www.goodeatsfanpage.com' + raw
        else:
            url = 'http://www.goodeatsfanpage.com' + raw
        tran_links.append(url)
    except:
        pass

In [39]:
url = 'https://www.seriouseats.com/recipes/2016/09/shakshuka-north-african-shirred-eggs-tomato-pepper-recipe.html'
driver.get(url)

In [67]:

html = driver.page_source
se_soup = bs(html, 'html.parser')
script_divs = se_soup.find_all('script', {'type': 'application/ld+json'})
j = json.loads(script_divs[11].text)
pprint(j)

{'@context': 'http://schema.org/',
 '@type': 'Recipe',
 'aggregateRating': {'@type': 'AggregateRating',
                     'ratingCount': '9',
                     'ratingValue': '4.6666666666667'},
 'author': {'@type': 'Person',
            'description': 'J. Kenji López-Alt is the Chief Culinary Advisor '
                           'of Serious Eats, and author of the James Beard '
                           'Award-nominated column The Food Lab, where he '
                           'unravels the science of home cooking. A '
                           'restaurant-trained chef and former Editor at '
                           "Cook's Illustrated magazine, his first book, The "
                           'Food Lab: Better Home Cooking Through Science is a '
                           'New York Times Best-Seller, the recipient of a '
                           'James Beard Award, and was named Cookbook of the '
                           'Year in 2015 by the International Association o

In [82]:

recipe_dict = {}
recipe_dict.setdefault('category_trees', [])
recipe_dict.setdefault('recipe', {})
for div in script_divs:
    j = json.loads(div.text)
    if j['@type'] == 'BreadcrumbList':
        category_tree = []
        for item in j['itemListElement']:
            category_tree.append(item['item']['name'])
        recipe_dict['category_trees'].append(category_tree)
    if j['@type'] == 'Recipe':
        rating_count = j['aggregateRating']['ratingCount']
        rating_value = j['aggregateRating']['ratingValue']
        author = j['author']['name']
        job_title = j['author']['jobTitle']
        description = j['description']
        keywords = j['keywords']
        name = j['name']
        categories = j['recipeCategory']
        cuisine = j['recipeCuisine']
        ingredients = j['recipeIngredient']
        steps = {}
        for i, step in enumerate(j['recipeInstructions']):
            this = 'step_' + str(i+1)
            steps[this] = step['text']
#             steps.append(step['text'])
        recipe_yield = j['recipeYield']
        time = str(isodate.parse_duration(j['totalTime']).total_seconds()/60)
        recipe_dict['recipe'] = {
            'name': name, 'rating_count': rating_count, 'rating_value': rating_value, 'author': author,
            'job_title': job_title, 'description': description, 'keywords': keywords, 'categories': categories,
            'cuisine': cuisine, 'ingredients': ingredients, 'steps': steps, 'recipe_yield': recipe_yield, 'time':time
        }


In [83]:
pprint(recipe_dict)

{'category_trees': [['Recipes', 'Ingredient', 'Vegetables', 'Tomato'],
                    ['Recipes', 'Ingredient', 'Vegetables', 'Other Vegetables'],
                    ['Recipes', 'Ingredient', 'Eggs'],
                    ['Recipes', 'Cuisine', 'African', 'Moroccan/North African'],
                    ['Recipes', 'Meal', 'Vegetarian'],
                    ['Recipes', 'Meal', 'Soups and Stews'],
                    ['Recipes', 'Meal', 'Quick and Easy'],
                    ['Recipes', 'Meal', 'Mains'],
                    ['Recipes', 'Meal', 'Breakfast and Brunch'],
                    ['Recipes', 'Method', 'Stovetop']],
 'recipe': {'author': 'J. Kenji López-Alt',
            'categories': ['Tomato',
                           'Other Vegetables',
                           'Eggs',
                           'Vegetarian',
                           'Soups and Stews',
                           'Quick and Easy',
                           'Mains',
                           'Breakfas